In [17]:
experience_salary = spark.sql(f"SELECT job_id,experience,salary FROM bing_lake_db.naukri_jobs WHERE lower(job_title) LIKE '%{job_titles}%'")
display(experience_salary)

StatementMeta(, a4ad5650-8f1b-4c75-bd00-b7d29482ffe4, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9bdebe75-5c39-4527-9065-c39529b803bc)

In [18]:
from pyspark.sql.functions import col, regexp_extract, when
# Define a regex pattern to extract numbers with optional commas and dots
regex_pattern = r'(\d+(?:\.\d+)?)(?:-(\d+(?:\.\d+)?))?'

# Extracting min_sal and max_sal
min_sal = regexp_extract(col("salary"), regex_pattern, 1)
max_sal = regexp_extract(col("salary"), regex_pattern, 2)

# Handling cases where no numbers are extracted (replace empty strings with 0)
experience_salary = experience_salary.withColumn("min_sal", when(min_sal == "", 0).otherwise(min_sal.cast("double"))) \
       .withColumn("max_sal", when(max_sal == "", 0).otherwise(max_sal.cast("double")))

experience_salary = experience_salary.drop("salary")

display(experience_salary)

StatementMeta(, a4ad5650-8f1b-4c75-bd00-b7d29482ffe4, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d3d76f05-e784-4616-8ee5-c1d17df5ef75)

In [19]:
# Define a regex pattern to extract numbers
regex_pattern = r'(\d+)(?:-(\d+))?'

# Extracting min_exp and max_exp
min_exp = regexp_extract(col("experience"), regex_pattern, 1)
max_exp = regexp_extract(col("experience"), regex_pattern, 2)

# Handling cases where no numbers are extracted (replace empty strings with 0)
experience_salary = experience_salary.withColumn("min_exp", when(min_exp == "", 0).otherwise(min_exp.cast("integer"))) \
       .withColumn("max_exp", when(max_exp == "", 0).otherwise(max_exp.cast("integer")))

experience_salary = experience_salary.drop("experience")

display(experience_salary)

StatementMeta(, a4ad5650-8f1b-4c75-bd00-b7d29482ffe4, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1a55be49-7219-48e6-809f-d174eb8bdf98)

In [20]:
from pyspark.sql.functions import col

exp_columns = [col('min_exp'), col('max_exp')]

averageFunc = sum(x for x in exp_columns)/len(exp_columns)

experience_salary=experience_salary.withColumn('avg_exp', averageFunc)

sal_columns = [col('min_sal'), col('max_sal')]

averageFunc = sum(x for x in sal_columns)/len(sal_columns)

experience_salary=experience_salary.withColumn('avg_sal', averageFunc)
display(experience_salary)

StatementMeta(, a4ad5650-8f1b-4c75-bd00-b7d29482ffe4, 22, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1ad4ba17-6344-44e1-b995-a130dfee1340)

In [21]:
from pyspark.sql import functions as F

experience_salary = experience_salary.withColumn("title_exp", 
                   F.when((F.col("max_exp") >= 0) & (F.col("max_exp") <= 1), "fresher")
                    .when((F.col("max_exp") >= 2) & (F.col("max_exp") < 3), "intermediate")
                    .when((F.col("max_exp") >=3) & (F.col("max_exp") <= 6), "mid-senior")
                    .otherwise("senior"))
display(experience_salary)

StatementMeta(, a4ad5650-8f1b-4c75-bd00-b7d29482ffe4, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 08acf3f1-c380-4353-9f8a-f83b6a01ba79)

In [22]:
from pyspark.sql.functions import lit
# Add a new column with a static value
experience_salary = experience_salary.withColumn('job_title', lit(f'{job_titles}'))
display(experience_salary)

StatementMeta(, a4ad5650-8f1b-4c75-bd00-b7d29482ffe4, 24, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 550fd9f3-5777-48e3-bffd-c7593a05efc5)

In [24]:
from pyspark.sql.utils import AnalysisException

try:
    table_name='bing_lake_db.experience_salary'
    experience_salary.write.format("delta").saveAsTable(table_name)
except AnalysisException:
    print("Table Already Exists")
    experience_salary.createOrReplaceTempView("experience_salary_new")
    spark.sql(f"""merge into {table_name} target_table
                using experience_salary_new source_view
                on source_view.job_id=target_table.job_id
                when matched and
                target_table.job_id<>source_view.job_id or
                target_table.min_exp<>source_view.min_exp or
                target_table.max_exp<>source_view.max_exp or
                target_table.avg_exp<>source_view.avg_exp or
                target_table.min_sal<>source_view.min_sal or
                target_table.max_sal<>source_view.max_sal or
                target_table.avg_sal<>source_view.avg_sal or
                target_table.title_exp<>source_view.title_exp or
                target_table.job_title<>source_view.job_title
                then update set *
                when not matched then insert *
                """)

StatementMeta(, a4ad5650-8f1b-4c75-bd00-b7d29482ffe4, 26, Finished, Available, Finished)

Table Already Exists


Py4JJavaError: An error occurred while calling o321.sql.
: org.apache.spark.sql.delta.DeltaUnsupportedOperationException: Cannot perform Merge as multiple source rows matched and attempted to modify the same
target row in the Delta table in possibly conflicting ways. By SQL semantics of Merge,
when multiple source rows match on the same target row, the result may be ambiguous
as it is unclear which source row should be used to update or delete the matching
target row. You can preprocess the source table to eliminate the possibility of
multiple matches. Please refer to
https://docs.delta.io/latest/delta-update.html#upsert-into-a-table-using-merge
	at org.apache.spark.sql.delta.DeltaErrorsBase.multipleSourceRowMatchingTargetRowInMergeException(DeltaErrors.scala:1102)
	at org.apache.spark.sql.delta.DeltaErrorsBase.multipleSourceRowMatchingTargetRowInMergeException$(DeltaErrors.scala:1099)
	at org.apache.spark.sql.delta.DeltaErrors$.multipleSourceRowMatchingTargetRowInMergeException(DeltaE